In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder, StandardScaler

from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

import seaborn as sns
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))


# Any results you write to the current directory are saved as output.

# 1. Load data:
- Load train and test set from csv files
- Split train into train and validation: 70 - 30

In [ ]:
train = pd.read_csv("../input/train.csv", index_col=0)
test = pd.read_csv("../input/test.csv", index_col=0)

pd.set_option("display.max_rows", 10)

In [ ]:
train

In [ ]:
test

# 2. Data understanding:

In [ ]:
# man or women survived more?
sns.barplot(x='Sex',y='Survived',data=train)

In [ ]:
# Pclass to survived?
sns.barplot(x='Pclass',y='Survived',data=train)

In [ ]:
# Embarked type? Which survived more
sns.barplot(x='Embarked', y='Survived', data=train)

In [ ]:
# Sibling types? which survived more
sns.barplot(x='SibSp', y='Survived', data=train)

In [ ]:
# Parch types? Which survived more
sns.barplot(x='Parch', y='Survived', data=train)

In [ ]:
def plot_(col, bins):
    

In [ ]:
temp['Fare'].max()

In [ ]:
a = pd.cut(temp['Fare'], bins=100)


In [ ]:
plt.figure(figsize=(10,10))
temp = train.copy()
temp['Fare_fact'] = a
#temp.groupby('Fare').mean()['Survived']
sns.barplot(x='Fare_fact',y='Survived', data=temp)

In [ ]:
train.loc[train['Survived']==1].groupby('Pclass').count()

In [ ]:
train['Ticket'].nunique()

In [ ]:
train['Ticket'].count()

In [ ]:
train.groupby('Ticket').mean().sort_values(by=['Survived'], ascending=False)['Survived']

In [ ]:
# correlation
sns.heatmap(train.corr())

In [ ]:
# see the imbalance of the data
n_class_1 = train.loc[train['Survived']==1, 'Survived'].count()
n_class_0 = train.loc[train['Survived']==0, 'Survived'].count()

print("Number of survived: ", n_survived)
print("Number of not survived: ", n_not_survived)

So we conclude that the data is imbalance between class 0 and 1

# 3. Data processing:
- First, deal with missing value
- 2 methods are used: LabelEncoder and OneHotEncoder for Logistic Regression and Decision Tree repectively

In [ ]:
# fillna
columns = train.columns
for col in columns:
    if(train[col].dtype == np.int64 or train[col].dtype == np.float64):
        train[col] = train[col].fillna(train[col].dropna().median())
    else:
        train[col] = train[col].fillna(train[col].dropna().mode()[0])
        
for col in test.columns:
    if(test[col].dtype == np.int64 or test[col].dtype == np.float64):
        test[col] = test[col].fillna(test[col].dropna().median())
    else:
        test[col] = test[col].fillna(test[col].dropna().mode()[0])
        
train

In [ ]:
# choose which columns to keep
# drop name columns

train_dropname = train.drop(columns=['Name'], axis=1)
test_dropname = test.drop(columns=['Name'], axis=1)

In [ ]:
# over sampling to reduce bad effect of imbalance class
train_data_1 = train_dropname.loc[train_dropname['Survived'] == 1].sample(n=n_class_0-n_class_1)
train_data_0 = train_dropname.loc[train_dropname['Survived'] == 0].sample(n=n_class_0)
train_data_ovs = pd.concat([train_dropname.loc[train_dropname['Survived'] == 1], train_data_1, train_data_0])

n_class_0_ovs = len(train_data_ovs.loc[train_data_ovs['Survived'] == 0])
n_class_1_ovs = len(train_data_ovs.loc[train_data_ovs['Survived'] == 1])

print("Number of class 1 after oversampling: ", n_class_1_ovs)
print("Number of class 0 after oversampling: ", n_class_0_ovs)

In [ ]:
# split train data and train target
train_data = train_data_ovs.drop(columns=['Survived'], axis=1)
train_target = train_data_ovs['Survived']
test_data = test_dropname

test_data


In [ ]:
# LabelEncoder
le = LabelEncoder()
train_test = pd.concat([train_data, test_data], axis=0)

train_data_le = train_data.copy()
test_data_le = train_data.copy()

categorical_cols = [ 'Sex', 'Ticket', 'Cabin', 'Embarked']
for col in categorical_cols:
    le.fit(train_test[col])
    train_data_le[col] = le.transform(train_data[col])
    test_data_le[col] = le.transform(train_data[col])
    
train_data_le

In [ ]:
sns.heatmap(train_data_le.corr())

In [ ]:
#One hot encoder
train_test = pd.concat([train_data, test_data])

#train_test = train_test.drop(columns=['Fare'], axis=1)
train_test_oh = pd.get_dummies(data=train_test)

train_data_oh = train_test_oh.iloc[:len(train_data),:]
test_data_oh = train_test_oh.iloc[len(train_data):,:]

train_data_oh.head()

In [ ]:
train_data_oh

# 4. Using the 3 model to classify
- LogisticRegression model
- DecisionTree model
- Support Vector Machine

In [ ]:
# linear model
# logistic regression
scaler = StandardScaler()
train_data_oh[['Age']] = scaler.fit_transform(train_data_oh[['Age']])

X_train, X_val, y_train, y_val = train_test_split(train_data_oh, train_target, test_size=0.3)

logistic_model = LogisticRegression(fit_intercept=True, C=100, penalty='l2',solver='newton-cg', 
                                    random_state=3)
logistic_model.fit(X_train, y_train)
train_acc = logistic_model.score(X_train, y_train)
val_acc = logistic_model.score(X_val, y_val)

print("train accuracy: ", train_acc)
print("validate accuracy: ", val_acc)

In [ ]:
# tree base model
X_train, X_val, y_train, y_val = train_test_split(train_data_le, train_target, test_size = 0.3)

dt_model = DecisionTreeClassifier(max_depth=10)
dt_model.fit(X_train, y_train)
train_acc = dt_model.score(X_train, y_train)
val_acc = dt_model.score(X_val, y_val)

print("train accuracy: ", train_acc)
print("validate accuracy: ", val_acc)

In [ ]:
# random forest
X_train, X_val, y_train, y_val = train_test_split(train_data_le, train_target, test_size = 0.3)

rf_model = RandomForestClassifier(max_features=6)
rf_model.fit(X_train, y_train)
train_acc = rf_model.score(X_train, y_train)
val_acc = rf_model.score(X_val, y_val)

print("train accuracy: ", train_acc)
print("validate accuracy: ", val_acc)

In [ ]:
# svm model
X_train, X_val, y_train, y_val = train_test_split(train_data_oh, train_target, test_size=0.3)

svm_model = SVC(kernel='linear')
svm_model.fit(X_train, y_train)
train_acc = svm_model.score(X_train, y_train)
val_acc = svm_model.score(X_val, y_val)

print("train accuracy: ", train_acc)
print("validate accuracy: ", val_acc)

# 5. Make prediction on test set

In [ ]:
# retrain on the whole training data
X = pd.concat([X_train, X_val])
y = pd.concat([y_train, y_val])
rf_model.fit(X, y)

In [ ]:
rf_model.score(X,y)

In [ ]:
prediction = svm_model.predict(test_data_oh)
submission = pd.DataFrame({'PassengerId':test.index, 'Survived':prediction})

In [ ]:
submission

In [ ]:
# save submission frame
submission.to_csv('submissionv10.csv',index=None)